## 3. Data Cleaning

Data: 12 de Junho de 2023

Esse notebook contém:


### Notas:



In [1]:
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from nbconf import *

from sentence_transformers import SentenceTransformer

import sqlite3

2023-08-10 11:02:16.327594: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 11:02:16.946193: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-10 11:02:16.946256: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-08-10 11:02:16.946261: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [70]:
# Load data saved in database
conn = sqlite3.connect(os.path.join(config.DATA_DIR, config.DATABASE))
cursor = conn.cursor()
df = pd.DataFrame(cursor.execute("SELECT * FROM metadata"))

### Data Selection

In [71]:
df.columns = [
    'url',
    'doi',
    'type',
    'author',
    'institute',
    'knowledge_area',
    'committee',
    'title_pt',
    'title_en',
    'keywords_pt',
    'keywords_en',
    'abstract_pt',
    'abstract_en',
    'publish_date'
]

In [72]:
df.head()

,url,doi,type,author,institute,knowledge_area,committee,title_pt,title_en,keywords_pt,keywords_en,abstract_pt,abstract_en,publish_date
0,https://www.teses.usp.br/teses/disponiveis/44/...,10.11606/T.44.2016.tde-06072016-171200,Tese de Doutorado,"Pinto, Irajá Damiani (Catálogo USP)",Instituto de Geociências,Área do Conhecimento,None,Corais carboníferos da Amazônia,Not available,Coral (gema orgânica) Estratigrafia - Amazônia...,Not available,O presente trabalho compreende o estudo de cor...,Not available,2016-07-07
1,https://www.teses.usp.br/teses/disponiveis/44/...,10.11606/T.44.2016.tde-06072016-172400,Tese de Doutorado,"Kawashita, Koji (Catálogo USP)",Instituto de Geociências,Área do Conhecimento,None,Metodo Rb-Sr em rochas sedimentares: aplicação...,Not available,Geocronologia Rochas sedimentares,Not available,Estudo radiométrico em sedimentos com posição ...,Not available,2016-07-07
2,https://www.teses.usp.br/teses/disponiveis/44/...,10.11606/D.44.2016.tde-06072016-172430,Dissertação de Mestrado,"Kawashita, Koji (Catálogo USP)",Instituto de Geociências,Geologia Geral,None,Um sensível espectrômetro de massa de fonte só...,Not available,Espectrometria de massas Geocronologia,Not available,Um espectrômetro de massa de fonte sólida de a...,Not available,2016-07-07
3,https://www.teses.usp.br/teses/disponiveis/44/...,10.11606/T.44.2016.tde-07072016-084400,Tese de Doutorado,"Szikszay, Mária (Catálogo USP)",Instituto de Geociências,Área do Conhecimento,None,Aspectos geoquímicos de rios da bacia do Paraná,Not available,Geoquímica - Bacia do Paraná,Not available,Os rios da bacia hidrográfica do Paraná drenam...,Not available,2016-07-07
4,https://www.teses.usp.br/teses/disponiveis/44/...,10.11606/T.44.2016.tde-07072016-085900,Tese de Doutorado,"Kanji, Milton Assis (Catálogo USP)",Instituto de Geociências,Área do Conhecimento,None,Resistência ao cisalhamento de contactos solo-...,Not available,Conservação do solo,Not available,Nos estudos referentes à estabilidade de maciç...,Not available,2016-07-07


#### Null values

In [73]:
import re

In [74]:
def clean_null(df):
    regex = re.compile("not\savailable|não\spossui|não\sdisponível|no\sabstract", flags=re.IGNORECASE)
    df = df.replace(regex, regex=True, value="")
    nonan_indexes = df[df['title_en'].notna() & df['abstract_en'].notnull()].index
    df = df.iloc[nonan_indexes,:]
    df = df.reset_index(drop=True)
    df = df.fillna("")
    assert df.isna().sum().sum() == 0
    
clean_null(df)

In [75]:
df[(df['doi'] == '10.11606/D.11.2018.tde-20181127-160125')]['title_en'].values[0]

'not available'

#### Author

In [47]:
def clean_author(df):
    def clean_string(s):
        return s.replace("(Catálogo USP)", "")
    df['author'] = df['author'].apply(clean_string)

### Saving

In [37]:
df.to_csv(os.path.join(config.DATA_DIR, "metadata.csv"), escapechar='\\', index=False)

### Loading

In [39]:
data = pd.read_csv(os.path.join(config.DATA_DIR, "csv/metadata.csv"), keep_default_na=False)

In [40]:
data.isna().sum()

url            0
type           0
author         0
institute      0
title_en       0
abstract_en    0
keywords_en    0
dtype: int64